In [50]:
import pandas as pd
import numpy as np
import pyodbc
from dotenv import load_dotenv
import os

In [51]:
load_dotenv()

Environment variables:
900439423aA


In [54]:
# Creating the database connection to run the SQL queries
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'
    'UID=sa;'
    f'PWD={os.getenv("PASSWORD")};'
    'DATABASE=HealthcareCostDatabase;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()

# Extract

In [3]:
HOSPITAL_SUBSET = ['Provider CCN','Hospital Name','State Code','County','City','Rural Versus Urban','Medicare CBSA Number','Medicaid Charges','Stand-Alone CHIP Charges','Total Costs','Cost To Charge Ratio','Net Income','Net Revenue from Medicaid','Net Revenue from Stand-Alone CHIP','Net Income from Service to Patients','Net Revenue from Medicaid','Total Other Expenses','Fiscal Year Begin Date','Fiscal Year End Date']
DEMOGRAPHIC_SUBSET = ['NAICS','NAICS_TITLE','H_PCT10','H_PCT25','H_MEDIAN','H_PCT75','H_PCT90','A_PCT10','A_PCT25','A_MEDIAN','A_PCT75','A_PCT90','H_MEAN','A_MEAN','AREA_TYPE','AREA_TITLE','OCC_CODE','OCC_TITLE','TOT_EMP','EMP_PRSE','OWN_CODE','PRIM_STATE']

In [4]:
# Creating methods to make the extraction process much easier
def read_hospital_data(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path)
    return df

def read_demographic_data(file_path: str) -> pd.DataFrame:
    df = pd.read_excel(file_path)
    return df

def get_subset(df: pd.DataFrame, subset: list) -> pd.DataFrame:
    return df[subset]

In [5]:
# Create an umbrella method for all the extraction processes in the ETL process
def extract(file_path: str, subset: list) -> pd.DataFrame:
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        df = pd.read_excel(file_path)
    df = df[subset]
    df.dropna(inplace=True)
    return df

In [6]:
df_hospital = extract('Hospital_Provider_Cost_Report_2021.csv', HOSPITAL_SUBSET)
df_hospital

,Provider CCN,Hospital Name,State Code,County,City,Rural Versus Urban,Medicare CBSA Number,Medicaid Charges,Stand-Alone CHIP Charges,Total Costs,Cost To Charge Ratio,Net Income,Net Revenue from Medicaid,Net Revenue from Stand-Alone CHIP,Net Income from Service to Patients,Net Revenue from Medicaid,Total Other Expenses,Fiscal Year Begin Date,Fiscal Year End Date
6,340141,NEW HANOVER REGIONAL MEDICAL CENTER,NC,NEW HANOVER,WILMINGTON,R,48900.0,"$144,283,620","$1,620,804","$330,818,093",0.23731,"$-16,912,668","$20,399,104","$232,178","$-35,863,488","$20,399,104","$225,130",2020-10-01,2021-01-31
61,341304,BERTIE MEMORIAL HOSPITAL,NC,BERTIE,WINDSOR,R,99934.0,"$4,085,177","$87,416","$19,032,247",0.429541,"$1,386,728","$1,867,178","$25,303","$3,202,075","$1,867,178","$2,804,488",2020-10-01,2021-09-30
62,341318,CHOWAN HOSPITAL INC.,NC,CHOWAN,EDENTON,R,99934.0,"$15,982,471","$191,756","$48,425,448",0.365642,"$13,507,530","$6,851,547","$53,779","$14,088,250","$6,851,547","$5,302,381",2020-10-01,2021-09-30
86,341324,THE OUTER BANKS HOSPITAL,NC,DARE,NAGS HEAD,R,99934.0,"$13,401,500","$239,650","$59,223,555",0.355038,"$26,591,135","$5,449,178","$69,415","$31,083,690","$5,449,178","$7,089,230",2020-10-01,2021-09-30
185,51315,TRINITY HOSPITAL,CA,TRINITY,WEAVERVILLE,R,99905.0,"$10,312,947","$27,188","$20,440,028",0.679888,"$10,700,717","$6,157,668","$5,670","$5,657,456","$6,157,668","$30,709",2021-01-01,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6023,450604,HILL COUNTRY MEMORIAL HOSPITAL,TX,GILLESPIE,FREDERICKSBURG,R,99945.0,"$9,209,150","$431,019","$68,425,263",0.268762,"$21,650,898","$3,109,039","$42,993","$-9,670,826","$3,109,039","$-1,032,122",2021-01-01,2021-12-31
6024,450634,MEDICAL CITY DENTON,TX,DENTON,DENTON,U,19124.0,"$132,583,639","$1,057,108","$161,809,885",0.069759,"$50,209,346","$6,508,700","$25,145","$49,821,219","$6,508,700",$-5,2021-01-01,2021-12-31
6025,450638,HOUSTON NORTHWEST MEDICAL CENTER,TX,HARRIS,HOUSTON,U,26420.0,"$664,569,226","$7,074,750","$272,129,988",0.076412,"$12,397,860","$51,596,209","$513,094","$11,326,074","$51,596,209",$-16,2021-06-01,2022-05-31
6026,450672,MEDICAL CITY FORT WORTH,TX,TARRANT,FORT WORTH,U,23104.0,"$197,488,655","$759,868","$263,958,551",0.084646,"$71,766,759","$11,649,167","$16,742","$71,139,780","$11,649,167",$8,2021-02-01,2022-01-31


In [7]:
df_demographic = extract('state_M2023_dl.xlsx', DEMOGRAPHIC_SUBSET)
df_demographic

,NAICS,NAICS_TITLE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,...,H_MEAN,A_MEAN,AREA_TYPE,AREA_TITLE,OCC_CODE,OCC_TITLE,TOT_EMP,EMP_PRSE,OWN_CODE,PRIM_STATE
0,0,Cross-industry,10.87,14.22,19.88,30.09,46.18,22620,29580,41350,...,25.67,53400,2,Alabama,00-0000,All Occupations,2053090,0,1235,AL
1,0,Cross-industry,24.38,35.18,47.95,67.22,95.44,50710,73180,99740,...,56.21,116920,2,Alabama,11-0000,Management Occupations,105580,0.8,1235,AL
2,0,Cross-industry,31.59,59.6,79.48,102.01,#,65700,123960,165320,...,106.26,221030,2,Alabama,11-1011,Chief Executives,720,6.8,1235,AL
3,0,Cross-industry,23.11,34.74,49.67,78.25,112.54,48080,72260,103320,...,62.17,129310,2,Alabama,11-1021,General and Operations Managers,34450,2.7,1235,AL
4,0,Cross-industry,*,*,*,*,*,18320,19670,24470,...,*,33690,2,Alabama,11-1031,Legislators,1140,9.1,1235,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37671,0,Cross-industry,14.79,14.79,18.25,18.74,20.48,30770,30770,37950,...,18.1,37650,3,Virgin Islands,53-7051,Industrial Truck and Tractor Operators,30,20.9,1235,VI
37672,0,Cross-industry,11.89,13.59,14.64,16.47,23.27,24740,28270,30460,...,15.38,31980,3,Virgin Islands,53-7061,Cleaners of Vehicles and Equipment,50,32.7,1235,VI
37673,0,Cross-industry,12.75,13.9,15.53,17.65,19.42,26520,28920,32300,...,15.98,33230,3,Virgin Islands,53-7062,"Laborers and Freight, Stock, and Material Move...",410,8.8,1235,VI
37674,0,Cross-industry,10.79,11,11.56,14.33,17.29,22450,22880,24040,...,13.18,27410,3,Virgin Islands,53-7064,"Packers and Packagers, Hand",100,13.7,1235,VI


# Transform

In [8]:
# Creating methods to make the extraction process much easier
def transform_hospital_data(df: pd.DataFrame) -> pd.DataFrame:
    # Need to transform the data
    df['Medicaid Charges'] = df['Medicaid Charges'].replace('[\$,]', '', regex=True).astype(float)
    df['Stand-Alone CHIP Charges'] = df['Stand-Alone CHIP Charges'].replace('[\$,]', '', regex=True).astype(float)
    df['Total Costs'] = df['Total Costs'].replace('[\$,]', '', regex=True).astype(float)
    df['Net Income'] = df['Net Income'].replace('[\$,]', '', regex=True).astype(float)
    df['Net Revenue from Medicaid'] = df['Net Revenue from Medicaid'].replace('[\$,]', '', regex=True).astype(float)
    df['Net Revenue from Stand-Alone CHIP'] = df['Net Revenue from Stand-Alone CHIP'].replace('[\$,]', '', regex=True).astype(float)
    df['Net Income from Service to Patients'] = df['Net Income from Service to Patients'].replace('[\$,]', '', regex=True).astype(float)
    df['Net Revenue from Medicaid'] = df['Net Revenue from Medicaid'].replace('[\$,]', '', regex=True).astype(float)
    df['Total Other Expenses'] = df['Total Other Expenses'].replace('[\$,]', '', regex=True).astype(float)
    
    # Renaming the ID column to match the other dataset
    df.rename(
        columns={
            "Provider CCN": "Hospital_ID",
            "Hospital Name": "Hospital_Name",
        }
    )
    # Removing duplicate columns that may have been created during the transformation process
    df = df.loc[:, ~df.columns.duplicated()]
    # Remove duplicate rows in the dataset that were made due to the data entry process where only the ID is made but the rest of the data is left empty or is the same exact record
    df.drop_duplicates(inplace=True)
    
    # Creating new columns that will be used for analysis
    df['Total Revenue'] = df['Net Income from Service to Patients'] + df['Net Revenue from Medicaid'] + df['Net Revenue from Stand-Alone CHIP']
    df['Profit Margin'] = df['Net Income from Service to Patients'] / df['Total Revenue']
    df['Cost To Revenue Ratio'] = df['Total Costs'] / df['Total Revenue']
    
    # Resetting the index
    df.reset_index(drop=True, inplace=True)
    
    return df

def transform_demographic_data(df: pd.DataFrame, year:int) -> pd.DataFrame:
    df.replace({'#': np.nan, "*": np.nan}, inplace=True)
    # df = df.apply(pd.to_numeric, errors='ignore')
    
    # Need to transform the data and fill the numeric columns with the median value
    for column in df.columns:
        if df[column].dtype == np.float64:
            df[column] = df[column].fillna(df[column].median())
            
    # Add a column for the year
    df['Year'] = year
    # Resetting the index
    df.reset_index(drop=True, inplace=True)
    
    return df

def create_surrogate_key(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    df[column_name] = range(1, len(df) + 1)
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    return df
    

In [9]:
df_hospital = transform_hospital_data(df_hospital)
df_hospital

C:\Users\Akhilesh Warty\AppData\Local\Temp\ipykernel_15704\3975306346.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


,Provider CCN,Hospital Name,State Code,County,City,Rural Versus Urban,Medicare CBSA Number,Medicaid Charges,Stand-Alone CHIP Charges,Total Costs,...,Net Income,Net Revenue from Medicaid,Net Revenue from Stand-Alone CHIP,Net Income from Service to Patients,Total Other Expenses,Fiscal Year Begin Date,Fiscal Year End Date,Total Revenue,Profit Margin,Cost To Revenue Ratio
0,340141,NEW HANOVER REGIONAL MEDICAL CENTER,NC,NEW HANOVER,WILMINGTON,R,48900.0,144283620.0,1620804.0,330818093.0,...,-16912668.0,20399104.0,232178.0,-35863488.0,225130.0,2020-10-01,2021-01-31,-15232206.0,2.354451,-21.718331
1,341304,BERTIE MEMORIAL HOSPITAL,NC,BERTIE,WINDSOR,R,99934.0,4085177.0,87416.0,19032247.0,...,1386728.0,1867178.0,25303.0,3202075.0,2804488.0,2020-10-01,2021-09-30,5094556.0,0.628529,3.735801
2,341318,CHOWAN HOSPITAL INC.,NC,CHOWAN,EDENTON,R,99934.0,15982471.0,191756.0,48425448.0,...,13507530.0,6851547.0,53779.0,14088250.0,5302381.0,2020-10-01,2021-09-30,20993576.0,0.671074,2.306679
3,341324,THE OUTER BANKS HOSPITAL,NC,DARE,NAGS HEAD,R,99934.0,13401500.0,239650.0,59223555.0,...,26591135.0,5449178.0,69415.0,31083690.0,7089230.0,2020-10-01,2021-09-30,36602283.0,0.849228,1.618029
4,51315,TRINITY HOSPITAL,CA,TRINITY,WEAVERVILLE,R,99905.0,10312947.0,27188.0,20440028.0,...,10700717.0,6157668.0,5670.0,5657456.0,30709.0,2021-01-01,2021-12-31,11820794.0,0.478602,1.729159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,450604,HILL COUNTRY MEMORIAL HOSPITAL,TX,GILLESPIE,FREDERICKSBURG,R,99945.0,9209150.0,431019.0,68425263.0,...,21650898.0,3109039.0,42993.0,-9670826.0,-1032122.0,2021-01-01,2021-12-31,-6518794.0,1.483530,-10.496614
357,450634,MEDICAL CITY DENTON,TX,DENTON,DENTON,U,19124.0,132583639.0,1057108.0,161809885.0,...,50209346.0,6508700.0,25145.0,49821219.0,-5.0,2021-01-01,2021-12-31,56355064.0,0.884059,2.871257
358,450638,HOUSTON NORTHWEST MEDICAL CENTER,TX,HARRIS,HOUSTON,U,26420.0,664569226.0,7074750.0,272129988.0,...,12397860.0,51596209.0,513094.0,11326074.0,-16.0,2021-06-01,2022-05-31,63435377.0,0.178545,4.289877
359,450672,MEDICAL CITY FORT WORTH,TX,TARRANT,FORT WORTH,U,23104.0,197488655.0,759868.0,263958551.0,...,71766759.0,11649167.0,16742.0,71139780.0,8.0,2021-02-01,2022-01-31,82805689.0,0.859117,3.187686


In [10]:
df_demographic = transform_demographic_data(df_demographic,2020)
df_demographic

C:\Users\Akhilesh Warty\AppData\Local\Temp\ipykernel_15704\3975306346.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'#': np.nan, "*": np.nan}, inplace=True)


,NAICS,NAICS_TITLE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,...,A_MEAN,AREA_TYPE,AREA_TITLE,OCC_CODE,OCC_TITLE,TOT_EMP,EMP_PRSE,OWN_CODE,PRIM_STATE,Year
0,0,Cross-industry,10.87,14.22,19.88,30.09,46.18,22620.0,29580.0,41350.0,...,53400.0,2,Alabama,00-0000,All Occupations,2053090,0,1235,AL,2020
1,0,Cross-industry,24.38,35.18,47.95,67.22,95.44,50710.0,73180.0,99740.0,...,116920.0,2,Alabama,11-0000,Management Occupations,105580,0.8,1235,AL,2020
2,0,Cross-industry,31.59,59.60,79.48,102.01,36.90,65700.0,123960.0,165320.0,...,221030.0,2,Alabama,11-1011,Chief Executives,720,6.8,1235,AL,2020
3,0,Cross-industry,23.11,34.74,49.67,78.25,112.54,48080.0,72260.0,103320.0,...,129310.0,2,Alabama,11-1021,General and Operations Managers,34450,2.7,1235,AL,2020
4,0,Cross-industry,17.87,21.00,24.74,30.51,36.90,18320.0,19670.0,24470.0,...,33690.0,2,Alabama,11-1031,Legislators,1140,9.1,1235,AL,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37671,0,Cross-industry,14.79,14.79,18.25,18.74,20.48,30770.0,30770.0,37950.0,...,37650.0,3,Virgin Islands,53-7051,Industrial Truck and Tractor Operators,30,20.9,1235,VI,2020
37672,0,Cross-industry,11.89,13.59,14.64,16.47,23.27,24740.0,28270.0,30460.0,...,31980.0,3,Virgin Islands,53-7061,Cleaners of Vehicles and Equipment,50,32.7,1235,VI,2020
37673,0,Cross-industry,12.75,13.90,15.53,17.65,19.42,26520.0,28920.0,32300.0,...,33230.0,3,Virgin Islands,53-7062,"Laborers and Freight, Stock, and Material Move...",410,8.8,1235,VI,2020
37674,0,Cross-industry,10.79,11.00,11.56,14.33,17.29,22450.0,22880.0,24040.0,...,27410.0,3,Virgin Islands,53-7064,"Packers and Packagers, Hand",100,13.7,1235,VI,2020


In [11]:
# Creating the dimension dataframes so that we can load them into the database

# Adding this function as a measure to ensure that the data keys are unique and dont clash with each other
def get_max_id(cursor: pyodbc.Cursor, table_name: str, column_name: str) -> int:
      query = f"SELECT MAX({column_name}) AS max_id FROM {table_name}"
      cursor.execute(query)
      result = cursor.fetchone()
      if result[0] is None:
         return 0
      return result[0]
      

def create_hospital_dimension(df: pd.DataFrame) -> pd.DataFrame:
   hospital_dim = pd.DataFrame({
      'Provider_CCN' : df['Provider CCN'],
      'Hospital_Name' : df['Hospital Name'],
   })
   
   hospital_dim = hospital_dim.drop_duplicates(subset=['Provider_CCN'])

   return hospital_dim

def create_geographic_info_dimension(df: pd.DataFrame) -> pd.DataFrame:
   geographic_info_dim = pd.DataFrame({
      'State_Code' : df['State Code'],
      'County' : df['County'],
      'City' : df['City'],
      'RuralVsUrban' : df['Rural Versus Urban'],
   })
   
   # Dropping duplicates in the dataset
   geographic_info_dim = geographic_info_dim.drop_duplicates(subset=['State_Code','County','City'])

   return geographic_info_dim

def create_date_dimension(df: pd.DataFrame) -> pd.DataFrame:
   date_dim = pd.DataFrame({
      # 'Date_Key' : range(1, 1+len(df['Fiscal Year Begin Date'].unique())),
      # 'Date' : pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'].unique())),
      'Year': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'].unique())).dt.year,
      'Quarter': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'].unique())).dt.quarter,
      'Month': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'].unique())).dt.month,
      'Day': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'].unique())).dt.day
   })
   
   date_dim.reset_index(drop=True, inplace=True)
   
   return date_dim

def create_occupational_info_dimension(df: pd.DataFrame) -> pd.DataFrame:
   occupational_info_dim = pd.DataFrame({
      # 'DemographicID': range(1, 1+len(df)),
      'NAICS' : df['NAICS'],
      'OccupationTitle' : df['OCC_TITLE'],
      'IndustryGroup' : df['OCC_CODE'],
      'AreaName': df['AREA_TITLE'],
      'AreaCode': df['PRIM_STATE'],
   })
   
   return occupational_info_dim

def create_financial_measurement(df: pd.DataFrame) -> pd.DataFrame:
   financial_measurement = pd.DataFrame({
   #  'ID' : range(1, 1+len(df)),
    'ProviderCCN': df['Provider CCN'],
    'MedicaidCharges': df['Medicaid Charges'],
    'StandAloneCHIPCharges': df['Stand-Alone CHIP Charges'],
    'TotalIncome': df['Total Revenue'],
    'Profit': df['Total Revenue'] - df['Total Costs'],
    'NetIncome': df['Net Income'],
    'Date': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'])),
    'StateCode': df['State Code'],
   })
   
   return financial_measurement

def create_demographic_measurement(df: pd.DataFrame) -> pd.DataFrame:
   demographic_measurement = pd.DataFrame({
      # 'ID' : range(1, 1+len(df)),
      'NAICS': df['NAICS'],
      'OccupationTitle': df['OCC_TITLE'],
      'AreaName': df['AREA_TITLE'],
      'AreaCode': df['PRIM_STATE'],
   })
   
   return demographic_measurement

def create_health_care_cost_staging_table(df: pd.DataFrame) -> pd.DataFrame:
   health_care_cost_staging_table = pd.DataFrame({
      # 'CostID': range(1, 1+len(df)),
      'Year': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'])).dt.year,
      'Quarter': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'])).dt.quarter,
      'Month': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'])).dt.month,
      'Day': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'])).dt.day,
      'HospitalName': df['Hospital Name'],
      'ProviderCCN': df['Provider CCN'],
      'StateCode': df['State Code'],
      'County': df['County'],
      'City': df['City'],
      'MedicaidCharges': df['Medicaid Charges'],
      'StandAloneCHIPCharges': df['Stand-Alone CHIP Charges'],
      'TotalCosts': df['Total Costs'],
      'NetIncome': df['Net Income'],
      'NetRevenueFromMedicaid': df['Net Revenue from Medicaid'],
      'NetRevenueFromStandAloneCHIP': df['Net Revenue from Stand-Alone CHIP'],
      'NetIncomeFromServiceToPatients': df['Net Income from Service to Patients'],
      'CostToChargeRatio': df['Cost To Charge Ratio'],
   })
   
   return health_care_cost_staging_table

def create_financial_measurement_staging_table(df: pd.DataFrame) -> pd.DataFrame:
   financial_staging_table = pd.DataFrame({
      'HospitalName' : df['Hospital Name'],
      'StateCode' : df['State Code'],
      'County' : df['County'],
      'City' : df['City'],
      'Year': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'])).dt.year,
      'Quarter': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'])).dt.quarter,
      'Day': pd.Series(pd.to_datetime(df['Fiscal Year Begin Date'])).dt.day,
      'TotalIncome': df['Total Revenue'],
      'Profit': df['Total Revenue'] - df['Total Costs'],
      'ProfitMargin': df['Profit Margin'],
   })
   
   return financial_staging_table

def create_demographic_measurement_staging_table(df: pd.DataFrame) -> pd.DataFrame:
   demographic_staging_table = pd.DataFrame({
      'NAICS': df['NAICS'],
      'OccupationTitle': df['OCC_TITLE'],
      'AreaName': df['AREA_TITLE'],
      'AreaCode': df['PRIM_STATE'],
      'Year': df['Year'],
      'MeanHourlyWage' : df['H_MEAN'],
      'MeanWage': df['A_MEAN'],
      'MedianAnnualWage': df['A_MEDIAN'],
      'PCT25AnnualWage': df['A_PCT25'],
      'PCT25HourlyWage': df['H_PCT25'],
      'PCT75AnnualWage': df['A_PCT75'],
      'PCT75HourlyWage': df['H_PCT75'],
   })
   
   return demographic_staging_table


def merge_date_dimension_to_financial_fact(fact_table: pd.DataFrame, date_dimension: pd.DataFrame) -> pd.DataFrame:
   fact_table = fact_table.merge(date_dimension[['Date']], left_on='Date', right_on='Date', how='inner')[['ProviderCCN','MedicaidCharges','StandAloneCHIPCharges','TotalIncome','Profit','NetIncome','Date']]
   return fact_table

def merge_health_care_cost_to_financial_fact(fact_table: pd.DataFrame, health_care_cost_dimension: pd.DataFrame) -> pd.DataFrame:
   fact_table = fact_table.merge(health_care_cost_dimension[['NetIncome','MedicaidCharges','StandAloneCHIPCharges']], how='left')[['CostID','Date_Key','ProviderCCN','TotalIncome','Profit']]
   return fact_table

def merge_occupational_info_to_demographic_measurement(demographic_measurement: pd.DataFrame, occupational_info_dimension: pd.DataFrame) -> pd.DataFrame:
   demographic_measurement = demographic_measurement.merge(occupational_info_dimension[['NAICS','OccupationTitle','IndustryGroup','AreaName','AreaCode','MeanWage','MeanHourlyWage','MedianAnnualWage','PCT25AnnualWage','PCT75AnnualWage','PCT25HourlyWage','PCT75HourlyWage']], how='left')[['OccupationTitle','AreaName','AreaCode']]
   return demographic_measurement
    

In [12]:
# Create an umbrella function that does all the transofrmations in one call
def transform_data(hospital_dataframe: pd.DataFrame, demographic_dataframe: pd.DataFrame,year:int) -> tuple:    
    df_hospital = transform_hospital_data(hospital_dataframe)
    df_demographic = transform_demographic_data(demographic_dataframe,year)
    
    # Creating Surrogate Keys for the dataframes
    df_hospital = create_surrogate_key(df_hospital, 'Hospital_ID')
    df_demographic = create_surrogate_key(df_demographic, 'Demographic_ID')
    
    # Now creating the dimension dataframes
    hospital_dim = create_hospital_dimension(df_hospital)
    date_dim = create_date_dimension(df_hospital)
    geographic_info_dimension = create_geographic_info_dimension(df_hospital)
    occupational_info_dim = create_occupational_info_dimension(df_demographic)
    
    # Creating the staging tables
    financial_staging_table = create_financial_measurement_staging_table(df_hospital)
    demographic_staging_table = create_demographic_measurement_staging_table(df_demographic)
    health_care_cost_staging_table = create_health_care_cost_staging_table(df_hospital)
    
    # Returning all the dimensions and fact tables for the ETL Process
    return hospital_dim, date_dim, occupational_info_dim,geographic_info_dimension,financial_staging_table,demographic_staging_table,health_care_cost_staging_table

## Load

In [13]:
# Creating the tables in the database, but needs to take care of initial load as well as incremental load
def check_if_tables_exist(cursor: pyodbc.Cursor,table_name: str) -> bool:
    # First we need to check if the the tables exist in the database
    query = f"""
        SELECT CASE WHEN EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}'
        ) THEN 1 ELSE 0 END AS Result
    """
    
    cursor.execute(query)
    result = cursor.fetchone()
    
    if result[0] == 1:
        return True
    else:
        return False

In [14]:
check_if_tables_exist(cursor, 'HospitalInfo')

True

In [15]:
# If the tables dont exist then the tables need to be created in the database
def create_hospital_dimension_table(cursor: pyodbc.Cursor) -> None:
    query = """
        CREATE TABLE HospitalInfo (
            HospitalID INT IDENTITY(1,1) PRIMARY KEY,
            Provider_CCN INT,
            Hospital_Name VARCHAR(255),
            EffectiveDate DATE DEFAULT GETDATE(), -- Creating the default date to the current date, will get changed when the record is updated
            ExpirationDate DATE DEFAULT '9999-12-31', -- Expiration date which will never be reached
            IsCurrent BIT DEFAULT 1 -- SCD Varaible to show if the record is the current record or not
        )
    """
    cursor.execute(query)
    cursor.commit()
    
    print('Hospital Dimension Table Created Successfully!')
    
def create_geographic_info_dimension_table(cursor: pyodbc.Cursor) -> None:
    query = """
        CREATE TABLE GeographicInfo (
            GeographicID INT IDENTITY(1,1) PRIMARY KEY,
            State_Code VARCHAR(10),
            County VARCHAR(255),
            City VARCHAR(255),
            RuralVsUrban VARCHAR(50),
            EffectiveDate DATE DEFAULT GETDATE(),
            ExpirationDate DATE DEFAULT '9999-12-31',
            IsCurrent BIT DEFAULT 1
        )
    
    """
    
    cursor.execute(query)
    cursor.commit()
    
def create_date_dimension_table(cursor: pyodbc.Cursor) -> None:
    query = """
        Create Table DateDimension (
            DateID INT IDENTITY(1,1) PRIMARY KEY,
            Year INT,
            Quarter INT,
            Month INT,
            Day INT,
        )
    """
    
    cursor.execute(query)
    cursor.commit()
    
    print('Date Dimension Table Created Successfully!')
    
def create_occupational_info_dimension_table(cursor: pyodbc.Cursor) -> None:
    query = """
        CREATE TABLE OccupationalInfo(
            DemographicID INT IDENTITY(1,1) PRIMARY KEY,
            NAICS INT,
            OccupationTitle VARCHAR(255),
            IndustryGroup VARCHAR(255),
            AreaName VARCHAR(255),
            AreaCode VARCHAR(10),
            EffectiveDate DATE DEFAULT GETDATE(),
            ExpirationDate DATE DEFAULT '9999-12-31',
            IsCurrent BIT DEFAULT 1
        )
    """
    cursor.execute(query)
    cursor.commit()
    
    print('Occupational Info Dimension Table Created Successfully!')
    
def create_financial_measurement_table(cursor: pyodbc.Cursor) -> None:
    query = """
        CREATE TABLE FinancialMeasurement(
            ID INT IDENTITY(1,1) PRIMARY KEY,
            HospitalID INT DEFAULT -1,
            GeographicID INT DEFAULT -1,
            Date_Key INT DEFAULT -1,
            TotalIncome FLOAT,
            Profit FLOAT,
            ProfitMargin FLOAT,
        )
    """

    cursor.execute(query)
    cursor.commit()
    
    print('Financial Measurement Table Created Successfully!')
    
def create_demographic_measurement_table(cursor: pyodbc.Cursor) -> None:
    query = """
        CREATE TABLE DemographicMeasurement(
            ID INT IDENTITY(1,1) PRIMARY KEY,
            DemographicID INT DEFAULT -1,
            DateID INT DEFAULT -1,
            MeanHourlyWage FLOAT,
            MeanWage FLOAT,
            MedianAnnualWage FLOAT,
            PCT25AnnualWage FLOAT,
            PCT25HourlyWage FLOAT,
            PCT75AnnualWage FLOAT,
            PCT75HourlyWage FLOAT,
        )
    """
    
    cursor.execute(query)
    cursor.commit()
    
    print('Demographic Measurement Table Created Successfully!')
    
def create_health_care_cost_table(cursor: pyodbc.Cursor) -> None:
    query = """
        CREATE TABLE HealthCareCost(
            ID INT IDENTITY(1,1) PRIMARY KEY,
            DateID INT DEFAULT -1,
            HospitalID INT DEFAULT -1,
            GeographicID INT DEFAULT -1,
            MedicaidCharges FLOAT,
            StandAloneCHIPCharges FLOAT,
            TotalCosts FLOAT,
            NetIncome FLOAT,
            NetRevenueFromMedicaid FLOAT,
            NetRevenueFromStandAloneCHIP FLOAT,
            NetIncomeFromServiceToPatients FLOAT,
            CostToChargeRatio FLOAT,
        )
    
    """
    
    cursor.execute(query)
    cursor.commit()
    
    print('Health Care Cost Table Created Successfully!')
    
def create_staging_table_for_financial_measurement(cursor: pyodbc.Cursor) -> None:
    query = """
            CREATE TABLE FinancialMeasurementStaging(
                HospitalName VARCHAR(255),
                StateCode VARCHAR(10),
                County VARCHAR(255),
                City VARCHAR(255),
                Year INT,
                Quarter INT,
                Day INT,
                TotalIncome FLOAT,
                Profit FLOAT,
                ProfitMargin FLOAT,
            )
    """
    cursor.execute(query)
    cursor.commit()
    
    print('Financial Measurement Staging Table Created Successfully!')
    
def create_staging_table_for_demographic_measurement(cursor: pyodbc.Cursor) -> None:
    query = """
            CREATE TABLE DemographicMeasurementStaging(
                NAICS INT,
                OccupationTitle VARCHAR(255),
                AreaName VARCHAR(255),
                AreaCode VARCHAR(10),
                Year INT,
                MeanHourlyWage FLOAT,
                MeanWage FLOAT,
                MedianAnnualWage FLOAT,
                PCT25AnnualWage FLOAT,
                PCT25HourlyWage FLOAT,
                PCT75AnnualWage FLOAT,
                PCT75HourlyWage FLOAT,
            )
    
    """
    
    cursor.execute(query)
    cursor.commit()
    
    print('Demographic Measurement Staging Table Created Successfully!')
    
def create_staging_table_for_health_care_cost(cursor: pyodbc.Cursor) -> None:
    query = """
        CREATE TABLE HealthCareCostStaging(
            Year INT,
            Quarter INT,
            Month INT,
            Day INT,
            ProviderCCN INT,
            HospitalName VARCHAR(255),
            StateCode VARCHAR(10),
            County VARCHAR(255),
            City VARCHAR(255),
            MedicaidCharges FLOAT,
            StandAloneCHIPCharges FLOAT,
            TotalCosts FLOAT,
            NetIncome FLOAT,
            NetRevenueFromMedicaid FLOAT,
            NetRevenueFromStandAloneCHIP FLOAT,
            NetIncomeFromServiceToPatients FLOAT,
            CostToChargeRatio FLOAT,
        )
    """
    
    cursor.execute(query)
    cursor.commit()
    
    print('Health Care Cost Staging Table Created Successfully!')
    

In [16]:
def verify_tables_in_database(cursor: pyodbc.Cursor) -> None:
    if not check_if_tables_exist(cursor, 'HospitalInfo'):
        create_hospital_dimension_table(cursor)
    
    if not check_if_tables_exist(cursor, 'DateDimension'):
        create_date_dimension_table(cursor)
    
    if not check_if_tables_exist(cursor, 'OccupationalInfo'):
        create_occupational_info_dimension_table(cursor)
    
    if not check_if_tables_exist(cursor, 'FinancialMeasurement'):
        create_financial_measurement_table(cursor)
    
    if not check_if_tables_exist(cursor, 'DemographicMeasurement'):
        create_demographic_measurement_table(cursor)
        
    if not check_if_tables_exist(cursor, 'HealthCareCost'):
        create_health_care_cost_table(cursor)
        
    if not check_if_tables_exist(cursor, 'GeographicInfo'):
        create_geographic_info_dimension_table(cursor)
        
    if not check_if_tables_exist(cursor, 'FinancialMeasurementStaging'):
        create_staging_table_for_financial_measurement(cursor)
        
    if not check_if_tables_exist(cursor, 'DemographicMeasurementStaging'):
        create_staging_table_for_demographic_measurement(cursor)
        
    if not check_if_tables_exist(cursor, 'HealthCareCostStaging'):
        create_staging_table_for_health_care_cost(cursor)

In [17]:
verify_tables_in_database(cursor)

In [80]:
# After veryfying the tables in the database, we can now load the data into the tables
def load_data_into_table(cursor: pyodbc.Cursor, table_name: str,df: pd.DataFrame, is_scd: bool = True,scd_num: int = 1) -> None:
    # First we need to check if the table is empty or not
    # If it is empty then it is initial load and there are no problems with all the SCD's
    # If it is not empty then staging tables need to be created and the SCD's need to be handled
    query = f"""
        SELECT COUNT(*) FROM {table_name}
    """
    
    cursor.execute(query)
    result = cursor.fetchone()
    
    if result[0] == 0:
        # This is initial load
        # We can directly load the data into the table
        query = f"""
            INSERT INTO {table_name} ({', '.join(df.columns)})
            VALUES ({','.join(['?']*len(df.columns))})
        """
        
        cursor.executemany(query, df.values.tolist())
        cursor.commit()
    else:
        # This is incremental load
        # We need to create the staging tables and handle the SCD's
        # Check which records are new and which records are updated
        if is_scd:
            if scd_num == 0:
                # SCD Type 0 is where the record is inserted and there are no changes allowed
                pass
            if scd_num == 1:
                # SCD Type 1 is where the record is overwritten
                # We need to overwrite the existing records in the table
                if table_name == 'DateDimension':
                    # Get all the records from the table
                    query = f"""
                        SELECT * FROM {table_name}
                    """
                    
                    cursor.execute(query)
                    columns = [column[0] for column in cursor.description]
                    result = cursor.fetchall()
                    
                    # Create a dataframe from the result
                    current_df = pd.DataFrame.from_records(result,columns=columns)
                    
                    # Check which records are new and which records are updated
                    # Check with Year,Quarter,Month,Day
                    df['unique_ID'] = df['Year'].astype(str)+"_"+df['Quarter'].astype(str)+"_"+df['Month'].astype(str)+"_"+df['Day'].astype(str)
                    current_df['unique_ID'] = current_df['Year'].astype(str)+"_"+current_df['Quarter'].astype(str)+"_"+current_df['Month'].astype(str)+"_"+current_df['Day'].astype(str)
                    
                    new_records = df[~df['unique_ID'].isin(current_df['unique_ID'])]
                    updated_records = df[df['unique_ID'].isin(current_df['unique_ID'])]
                    
                    new_records = new_records.drop(columns=['unique_ID'])
                    updated_records = updated_records.drop(columns=['unique_ID'])
                    
                    # Load the new records into the table
                    print("Records to be inserted into Date Dimension: ",len(new_records))
                    if len(new_records) > 0:
                        # If the new records are greater than 0 then we need to load the new records into the table else there is an error
                        for _, row in new_records.iterrows():
                            print(f'NEW DATE RECORD : YEAR: {row["Year"]}, QUARTER: {row["Quarter"]}, MONTH: {row["Month"]}, DAY: {row["Day"]}')
                        query = f"""
                            INSERT INTO {table_name} ({', '.join(new_records.columns)})
                            VALUES ({','.join(['?']*len(new_records.columns))})
                        """
                        
                        cursor.executemany(query, new_records.values.tolist())
                        cursor.commit()
                        
                    # Update the existing records in the table
                    print("Records to be updated in Date Dimension: ",len(updated_records))
                    if len(updated_records) > 0:
                        # Check if any columns have changed
                        for _, row in updated_records.iterrows():
                            # Need to check for year, quarter, month, day
                            if current_df[(current_df['Year'] == row['Year']) & (current_df['Quarter'] == row['Quarter']) & (current_df['Month'] == row['Month']) & (current_df['Day'] == row['Day'])].empty:
                                pass
                            else:
                                current_row = current_df[(current_df['Year'] == row['Year']) & (current_df['Quarter'] == row['Quarter']) & (current_df['Month'] == row['Month']) & (current_df['Day'] == row['Day'])].iloc[0]
                                changed_columns = []
                                for column in updated_records.columns:
                                    if row[column] != current_row[column]:
                                        changed_columns.append(column)
                                        
                                print(f'MATCHED DATE RECORD: YEAR: {row["Year"]}, QUARTER: {row["Quarter"]}, MONTH: {row["Month"]}, DAY: {row["Day"]}')
                                    
                                if len(changed_columns) > 0:
                                    # There are changes that need overwriting
                                    print(f'UPDATED RECORDS: YEAR: {row["Year"]}, QUARTER: {row["Quarter"]}, MONTH: {row["Month"]}, DAY: {row["Day"]}')
                                    update_query = f"""
                                        UPDATE {table_name}
                                        SET {', '.join([f'{column} = ?' for column in changed_columns])}
                                        WHERE Year = ? AND Quarter = ? AND Month = ? AND Day = ?
                                    """
                                
                                    cursor.execute(update_query, tuple(row[changed_columns]) + tuple(row[['Year','Quarter','Month','Day']]))
                                    cursor.commit()
                            
                    print("Updated Date Dimension Table Successfully!")
                
            if scd_num == 2:
                
                if table_name == 'HospitalInfo':
                    # Get all the records from the table
                    query = f"""
                        SELECT * FROM {table_name}
                    """
        
                    cursor.execute(query)
                    columns = [column[0] for column in cursor.description]
                    result = cursor.fetchall()
        
                    # Create a dataframe from the result
                    current_df = pd.DataFrame.from_records(result,columns=columns)
        
                    # Check which records are new and which records are updated
                    hospital_new_records = df[~df['Provider_CCN'].isin(current_df['Provider_CCN'])]
                    hospital_updated_records = df[df['Provider_CCN'].isin(current_df['Provider_CCN'])]
        
                    # Load the new records into the table
                    if len(hospital_new_records) > 0:
                        # If the new records are greater than 0 then we need to load the new records into the table else there is an error
                        for _, row in hospital_new_records.iterrows():
                            print(f'NEW HOSPITAL INFO RECORD : PROVIDER CCN: {row["Provider_CCN"]}, HOSPITAL NAME: {row["Hospital_Name"]}')
                            
                        query = f"""
                            INSERT INTO {table_name} ({', '.join(hospital_new_records.columns)})
                            VALUES ({','.join(['?']*len(hospital_new_records.columns))})
                        """
        
                        cursor.executemany(query, hospital_new_records.values.tolist())
        
                        cursor.commit()
        
                    # Update the existing records in the table
                    # We need to handle the SCD's here
                    # We need to set the IsCurrent to 0 for the existing records
                    # We need to set the ExpirationDate to the current date for the existing records
                    # We need to insert the updated records as new records in the table
                    # We need to set the EffectiveDate to the current date for the updated records
                    # We need to set the IsCurrent to 1 for the updated records
                    # We need to set the ExpirationDate to '9999-12-31' for the updated records
                    if len(hospital_updated_records) > 0:
                        # Check if any columns have changed
                        added_records = []
                        for _, row in hospital_updated_records.iterrows():
                            # Need to check for provider ccn
                            current_row = current_df[current_df['Provider_CCN'] == row['Provider_CCN']].iloc[0]
                            if current_row.empty:
                                continue
                            else:
                                print(f'MATCHED HOSPITAL INFO RECORD : PROVIDER CCN: {row["Provider_CCN"]}, HOSPITAL NAME: {row["Hospital_Name"]} , MATCHED PROVIDER CCN: {current_row["Provider_CCN"]}, HOSPITAL NAME: {current_row["Hospital_Name"]}')
                                changed_columns = []
                                for column in hospital_updated_records.columns:
                                    if row[column] != current_row[column]:
                                        changed_columns.append(column)
        
                                if len(changed_columns) > 0:
                                    # First Making the Current Bit 0 and Expiring the record
                                    print(f'UPDATED HOSPITAL INFO RECORD : PROVIDER CCN: {row["Provider_CCN"]}, HOSPITAL NAME: {row["Hospital_Name"]}, CHANGED COLUMNS: {changed_columns}, MATCHED PROVIDER CCN: {current_row["Provider_CCN"]}, HOSPITAL NAME: {current_row["Hospital_Name"]}')
                                    update_query = f"""
                                        UPDATE {table_name}
                                        SET IsCurrent = 0, ExpirationDate = GETDATE()
                                        WHERE Provider_CCN = ?
                                    """
                                    cursor.execute(update_query, (row['Provider_CCN'],))
                                    cursor.commit()
        
                                    # Insert the updated record with new data
                                    insert_query = f"""
                                        INSERT INTO {table_name} ({', '.join(hospital_updated_records.columns)})
                                        VALUES ({','.join(['?']*len(hospital_updated_records.columns))})
                                    """
                                    cursor.execute(insert_query, tuple(row))
                                    cursor.commit()
                                
                                    # Add this updated row to the added records dataframe
                                    added_records.append(row)
                        
                        hospital_updated_records = pd.DataFrame(added_records,columns=hospital_updated_records.columns)
                        print('Records Updated in HospitalInfo Dimension: ',len(hospital_updated_records))
                        
                                        
                    print("Updated Hospital Dimension Table Successfully!")
                elif table_name == 'GeographicInfo':
                    # Get all the records from the table
                    query = f"""
                        SELECT * FROM {table_name}
                    """
                    
                    cursor.execute(query)
                    columns = [column[0] for column in cursor.description]
                    result = cursor.fetchall()
                    
                    # Create a dataframe from the result
                    current_df = pd.DataFrame.from_records(result,columns=columns)
                    
                    # Check which records are new and which records are updated
                    df['unique_ID'] = df['State_Code']+"_"+df['County']+"_"+df['City']
                    current_df['unique_ID'] = current_df['State_Code']+"_"+current_df['County']+"_"+current_df['City']
                    
                    geographic_new_records = df[~df['unique_ID'].isin(current_df['unique_ID'])]
                    geographic_updated_records = df[df['unique_ID'].isin(current_df['unique_ID'])]
                    
                    geographic_new_records = geographic_new_records.drop(columns=['unique_ID'])
                    geographic_updated_records = geographic_updated_records.drop(columns=['unique_ID'])
                    
                    geographic_updated_records = geographic_updated_records[['State_Code','County','City','RuralVsUrban']]
                    
                    # Load the new records into the table
                    if len(geographic_new_records) > 0:
                        # If the new records are greater than 0 then we need to load the new records into the table else there is an error
                        for _, row in geographic_new_records.iterrows():
                            print(f'NEW RECORD : STATE_CODE: {row["State_Code"]}, COUNTY: {row["County"]}, CITY: {row["City"]}, RURAL VS URBAN: {row["RuralVsUrban"]}')
                            
                        query = f"""
                            INSERT INTO {table_name} ({', '.join(geographic_new_records.columns)})
                            VALUES ({','.join(['?']*len(geographic_new_records.columns))})
                        """
                        
                        cursor.executemany(query, geographic_new_records.values.tolist())
                        cursor.commit()
                        
                    # Update the existing records in the table
                    # We need to handle the SCD's here
                    # We need to set the IsCurrent to 0 for the existing records
                    # We need to set the ExpirationDate to the current date for the existing records
                    # We need to insert the updated records as new records in the table
                    # We need to set the EffectiveDate to the current date for the updated records
                    # We need to set the IsCurrent to 1 for the updated records
                    # We need to set the ExpirationDate to '9999-12-31' for the updated records
                    if len(geographic_updated_records) > 0:
                        # Check if any columns have changed
                        added_records = []
                        for _, row in geographic_updated_records.iterrows():
                            # Need to check for state code, county, city, rural vs urban
                            
                            # We need to check all columns of the geographic info table
                            current_row = current_df.loc[(current_df['State_Code'] == row['State_Code']) & (current_df['County'] == row['County']) & (current_df['City'] == row['City'])]
                            if current_row.empty:
                                continue
                            else:  
                                current_row = current_row.iloc[0]
                                changed_columns = [col for col in geographic_updated_records.columns if row[col] != current_row[col]]
                                print(f'MATCHED RECORD : Row State Code: {row["State_Code"]}, County: {row["County"]}, City: {row["City"]}, Changed Columns: {changed_columns}, Matched Row State Code: {current_row["State_Code"]}, County: {current_row["County"]}, City: {current_row["City"]}')
                                if len(changed_columns) > 0:
                                    # First Making the Current Bit 0 and Expiring the record
                                    print(f'UPDATED : Row State Code: {row["State_Code"]}, County: {row["County"]}, City: {row["City"]}, Changed Columns: {changed_columns}, Matched Row State Code: {current_row["State_Code"]}, County: {current_row["County"]}, City: {current_row["City"]}')
                                    update_query = f"""
                                        UPDATE {table_name}
                                        SET IsCurrent = 0, ExpirationDate = GETDATE()
                                        WHERE State_Code = ? AND County = ? AND City = ?
                                    """
                                    cursor.execute(update_query, (row['State_Code'],row['County'],row['City']))
                                    cursor.commit()
                                
                                    # Insert the updated record with new data
                                    insert_query = f"""
                                        INSERT INTO {table_name} ({', '.join(geographic_updated_records.columns)})
                                        VALUES ({','.join(['?']*len(geographic_updated_records.columns))})
                                    """
                                    cursor.execute(insert_query, tuple(row))
                                    cursor.commit()
                                    added_records.append(row)
                                
                        geographic_updated_records = pd.DataFrame(added_records,columns=geographic_updated_records.columns)
                        print('Records Updated in GeographicInfo Dimension: ',len(geographic_updated_records))
                                
                    print("Updated Geographic Info Dimension Table Successfully!")    
                elif table_name == 'OccupationalInfo':
                    # Get all the records from the table
                    query = f"""
                        SELECT * FROM {table_name}
                    """
                    
                    cursor.execute(query)
                    columns = [column[0] for column in cursor.description]
                    result = cursor.fetchall()
                    
                    # Create a dataframe from the result
                    current_df = pd.DataFrame.from_records(result,columns=columns)
                    
                    # Check which records are new and which records are updated
                    current_df['unique_ID'] = current_df['NAICS'].astype(str)+"_"+current_df['OccupationTitle']+"_"+current_df['AreaName']+"_"+current_df['AreaCode']
                    df['unique_ID'] = df['NAICS'].astype(str)+"_"+df['OccupationTitle']+"_"+df['AreaName']+"_"+df['AreaCode']
                   
                    occupational_new_records = df[~df['unique_ID'].isin(current_df['unique_ID'])]
                    occupational_updated_records = df[df['unique_ID'].isin(current_df['unique_ID'])]
                    
                    occupational_new_records = occupational_new_records.drop(columns=['unique_ID'])
                    occupational_updated_records = occupational_updated_records.drop(columns=['unique_ID'])
                    
                    occupational_updated_records = occupational_updated_records[['NAICS','OccupationTitle','IndustryGroup','AreaName','AreaCode']]
                    
                    # Load the new records into the table
                    if len(occupational_new_records) > 0:
                        # If the new records are greater than 0 then we need to load the new records into the table else there is an error
                        for _, row in occupational_new_records.iterrows():
                            print(f'NEW OCCUPATION INFO RECORD : NAICS: {row["NAICS"]}, OCCUPATION TITLE: {row["OccupationTitle"]}, INDUSTRY GROUP: {row["IndustryGroup"]}, AREA NAME: {row["AreaName"]}, AREA CODE: {row["AreaCode"]}')
                        query = f"""
                            INSERT INTO {table_name} ({', '.join(occupational_new_records.columns)})
                            VALUES ({','.join(['?']*len(occupational_new_records.columns))})
                        """
                        
                        cursor.executemany(query, occupational_new_records.values.tolist())
                        cursor.commit()
                        
                    # Update the existing records in the table
                    # We need to handle the SCD's here
                    # We need to set the IsCurrent to 0 for the existing records
                    # We need to set the ExpirationDate to the current date for the existing records
                    # We need to insert the updated records as new records in the table
                    # We need to set the EffectiveDate to the current date for the updated records
                    # We need to set the IsCurrent to 1 for the updated records
                    # We need to set the ExpirationDate to '9999-12-31' for the updated records
                    if len(occupational_updated_records) > 0:
                        # Check if any columns have changed
                        added_records = []
                        for _, row in occupational_updated_records.iterrows():
                            # Need to check for state code, county, city, rural vs urban
                            
                            # There is no unique key in the occupational info table so we need to check all the columns
                            current_row = current_df[(current_df['NAICS'] == row['NAICS']) & (current_df['OccupationTitle'] == row['OccupationTitle']) & (current_df['AreaName'] == row['AreaName']) & (current_df['AreaCode'] == row['AreaCode'])]
                            if current_row.empty:
                                continue
                            else:
                                current_row = current_row.iloc[0]
                                changed_columns = []
                                for column in occupational_updated_records.columns:
                                    if row[column] != current_row[column]:
                                        changed_columns.append(column)
                                    
                                print(f'MATCHED OCCUPATION INFO RECORD : NAICS: {row["NAICS"]}, OCCUPATION TITLE: {row["OccupationTitle"]}, INDUSTRY GROUP: {row["IndustryGroup"]}, AREA NAME: {row["AreaName"]}, AREA CODE: {row["AreaCode"]}')
                                if len(changed_columns) > 0:
                                    # First Making the Current Bit 0 and Expiring the record
                                    print(f'UPDATED OCCUPATION INFO RECORD : NAICS: {row["NAICS"]}, OCCUPATION TITLE: {row["OccupationTitle"]}, INDUSTRY GROUP: {row["IndustryGroup"]}, AREA NAME: {row["AreaName"]}, AREA CODE: {row["AreaCode"]}')
                                    update_query = f"""
                                        UPDATE {table_name}
                                        SET IsCurrent = 0, ExpirationDate = GETDATE()
                                        WHERE NAICS = ?
                                    """
                                    cursor.execute(update_query, (row['NAICS'],))
                                    cursor.commit()
                                
                                    # Insert the updated record with new data
                                    insert_query = f"""
                                        INSERT INTO {table_name} ({', '.join(occupational_updated_records.columns)})
                                        VALUES ({','.join(['?']*len(occupational_updated_records.columns))})
                                    """
                                    cursor.execute(insert_query, tuple(row))
                                    cursor.commit()
                                
                                    added_records.append(row)
                                
                        occupational_updated_records = pd.DataFrame(added_records,columns=occupational_updated_records.columns)
                        print('Records Updated in OccupationalInfo Dimension: ',len(occupational_updated_records))
                
                if table_name == 'HospitalInfo':
                    return hospital_new_records,hospital_updated_records
                elif table_name == 'GeographicInfo':
                    return geographic_new_records,geographic_updated_records
                elif table_name == 'OccupationalInfo':
                    return occupational_new_records,occupational_updated_records 
                                
   
                    

def load_data_into_staging_tables(cursor: pyodbc.Cursor, table_name: str, df: pd.DataFrame) -> None:
    # Now we need to load the data into the staging tables
    # This will always be empty when loading the data
    
    # Delete existing records just in case to make sure there are no overlaps in the Staging table
    query = f"""
        DELETE FROM {table_name}
    """
    
    cursor.execute(query)
    cursor.commit()
    
    query = f"""
        INSERT INTO {table_name} ({', '.join(df.columns)})
        VALUES ({','.join(['?']*len(df.columns))})
    """
    
    cursor.executemany(query, df.values.tolist())
    cursor.commit()
    
    
def delete_data_from_table(cursor: pyodbc.Cursor, table_name: str) -> None:
    query = f"""
        DELETE FROM {table_name}
    """
    
    cursor.execute(query)
    cursor.commit()
    
    print(f'Deleted all records from {table_name} successfully!')

In [81]:
def load_data_into_database(cursor: pyodbc.Cursor, hospitalInfo_dim: pd.DataFrame,date_dim: pd.DataFrame, occupationInfo_dim: pd.DataFrame,geographic_info_dim,update: bool = False):
    # Loading the data into the database
    if not update:
        load_data_into_table(cursor, 'HospitalInfo', hospitalInfo_dim)
        load_data_into_table(cursor, 'DateDimension', date_dim)
        load_data_into_table(cursor, 'OccupationalInfo', occupationInfo_dim)
        load_data_into_table(cursor, 'GeographicInfo', geographic_info_dim)
    else:
        hospital_new_records,hospital_updated_records  = load_data_into_table(cursor, 'HospitalInfo', hospitalInfo_dim, True, 2)
        load_data_into_table(cursor, 'DateDimension', date_dim)
        occupational_new_records,occupational_updated_records = load_data_into_table(cursor, 'OccupationalInfo', occupationInfo_dim,True,2)
        geographic_new_records,geographic_updated_records = load_data_into_table(cursor, 'GeographicInfo', geographic_info_dim,True,2)
        return hospital_new_records,hospital_updated_records,occupational_new_records,occupational_updated_records,geographic_new_records,geographic_updated_records
    
    print('Loaded all the data into the database successfully!')

    
def delete_data_from_database(cursor: pyodbc.Cursor) -> None:
    delete_data_from_table(cursor, 'HospitalInfo')
    delete_data_from_table(cursor, 'DateDimension')
    delete_data_from_table(cursor, 'OccupationalInfo')
    delete_data_from_table(cursor, 'GeographicInfo')
    delete_data_from_table(cursor, 'FinancialMeasurement')
    delete_data_from_table(cursor, 'DemographicMeasurement')
    delete_data_from_table(cursor, 'HealthCareCost')
    delete_data_from_table(cursor, 'FinancialMeasurementStaging')
    delete_data_from_table(cursor, 'DemographicMeasurementStaging')
    delete_data_from_table(cursor, 'HealthCareCostStaging')
    
def generate_fact_tables(cursor: pyodbc.Cursor) -> None:
    
    delete_query = """
        DELETE FROM FinancialMeasurement
    """
    
    cursor.execute(delete_query)
    cursor.commit()
    
    query = """
        WITH InterDate AS (
            SELECT 
		        DateID,
		        HospitalName,
		        StateCode,
		        County,
		        City,
		        TotalIncome, 
		        Profit, 
		        ProfitMargin,
		        ROW_NUMBER() OVER (PARTITION BY HospitalName ORDER BY DateDimension.DateID DESC) AS RowNum
            FROM FinancialMeasurementStaging
            INNER JOIN DateDimension
            ON DateDimension.Year = FinancialMeasurementStaging.Year
        ),
        InterHospital AS (
            SELECT 
		        HospitalInfo.HospitalID,
		        DateID as Date_Key,
		        StateCode,
		        County,
		        City,
		        TotalIncome,
		        Profit,
		        ProfitMargin 
            FROM (SELECT DateID,HospitalName,StateCode,County,City,TotalIncome,Profit,ProfitMargin FROM InterDate WHERE RowNum = 1) as t1
            INNER JOIN HospitalInfo
            ON t1.HospitalName = HospitalInfo.Hospital_Name AND HospitalInfo.IsCurrent = 1
        )

        SELECT HospitalID,GeographicID,Date_Key,TotalIncome,Profit,ProfitMargin FROM InterHospital
        INNER JOIN GeographicInfo
        ON InterHospital.StateCode = GeographicInfo.State_Code
    """
    
    cursor.execute(query)
    results = cursor.fetchall()
    
    insert_query = """
        INSERT INTO FinancialMeasurement (HospitalID,GeographicID,Date_Key,TotalIncome,Profit,ProfitMargin)
        VALUES (?,?,?,?,?,?)
    """
    
    cursor.executemany(insert_query, results)
    cursor.commit()
    
    delete_query = """
        DELETE FROM DemographicMeasurement
    """
    
    cursor.execute(delete_query)
    cursor.commit()
    
    query = """
        WITH DemographicInter AS (
            SELECT 
	            DemographicID,
	            DemographicMeasurementStaging.Year,
	            DemographicMeasurementStaging.MeanHourlyWage,
	            DemographicMeasurementStaging.MeanWage,
	            DemographicMeasurementStaging.MedianAnnualWage,
	            DemographicMeasurementStaging.PCT25AnnualWage, 
	            DemographicMeasurementStaging.PCT25HourlyWage,
	            DemographicMeasurementStaging.PCT75AnnualWage,
	            DemographicMeasurementStaging.PCT75HourlyWage 
            FROM DemographicMeasurementStaging
            INNER JOIN OccupationalInfo
            ON OccupationalInfo.NAICS = DemographicMeasurementStaging.NAICS AND OccupationalInfo.OccupationTitle = DemographicMeasurementStaging.OccupationTitle AND OccupationalInfo.AreaName = DemographicMeasurementStaging.AreaName AND OccupationalInfo.AreaCode = DemographicMeasurementStaging.AreaCode
        ),
        DateDemographicInter AS (
            SELECT 
	            DemographicInter.DemographicID,
	            DateDimension.DateID,
	            DemographicInter.MeanHourlyWage,
	            DemographicInter.MeanWage,
	            DemographicInter.MedianAnnualWage,
	            DemographicInter.PCT25AnnualWage, 
	            DemographicInter.PCT25HourlyWage,
	            DemographicInter.PCT75AnnualWage,
	            DemographicInter.PCT75HourlyWage,
	            ROW_NUMBER() OVER (PARTITION BY DemographicID ORDER BY DateDimension.DateID DESC) AS RowNum 
        FROM DemographicInter
        INNER JOIN DateDimension
        ON DemographicInter.Year = DateDimension.Year
        )

        SELECT DemographicID,DateID,MeanHourlyWage,MeanWage,MedianAnnualWage,PCT25AnnualWage,PCT25HourlyWage,PCT75AnnualWage,PCT75HourlyWage FROM DateDemographicInter
        WHERE RowNum = 1
    """
    
    cursor.execute(query)
    results = cursor.fetchall()
    
    insert_query = """
        INSERT INTO DemographicMeasurement (DemographicID,DateID,MeanHourlyWage,MeanWage,MedianAnnualWage,PCT25AnnualWage,PCT25HourlyWage,PCT75AnnualWage,PCT75HourlyWage)
        VALUES (?,?,?,?,?,?,?,?,?)
    """
    
    cursor.executemany(insert_query, results)
    cursor.commit()
    
    delete_query = """
        DELETE FROM HealthCareCost
    """
    
    cursor.execute(delete_query)
    cursor.commit()
    
    query = """
            WITH HealthCareCostHospitalInter AS (
                SELECT 
                HospitalID,
                Year,
                Quarter,
                Month,
                Day,
                StateCode,
                County,City,MedicaidCharges,StandAloneCHIPCharges,TotalCosts,NetIncome,NetRevenueFromMedicaid,NetRevenueFromStandAloneCHIP,NetIncomeFromServiceToPatients, CostToChargeRatio 
            FROM HealthCareCostStaging
            INNER JOIN HospitalInfo
            ON HospitalInfo.Provider_CCN = HealthCareCostStaging.ProviderCCN
            ),
            HealthCareCostGeographicInter AS (
                SELECT 
                HospitalID,
                GeographicID,
                Year,
                Month,
                Quarter,
                MedicaidCharges,
                StandAloneCHIPCharges,
                TotalCosts,
                NetIncome,
                NetRevenueFromMedicaid,
                NetRevenueFromStandAloneCHIP,
                NetIncomeFromServiceToPatients,
                CostToChargeRatio 
            FROM HealthCareCostHospitalInter
            INNER JOIN GeographicInfo
            ON HealthCareCostHospitalInter.StateCode = GeographicInfo.State_Code
            ),
            HealthCareDateInter AS (
                SELECT 
                HospitalID,
                GeographicID,
                DateID,
                ROW_NUMBER() OVER (PARTITION BY HospitalID ORDER BY DateID DESC) as RowNumber,
                MedicaidCharges,
                StandAloneCHIPCharges,
                TotalCosts,
                NetIncome,
                NetRevenueFromMedicaid,
                NetRevenueFromStandAloneCHIP,
                NetIncomeFromServiceToPatients,
                CostToChargeRatio 
            FROM HealthCareCostGeographicInter
            INNER JOIN DateDimension
            ON DateDimension.Year = HealthCareCostGeographicInter.Year
            )

            SELECT HospitalID,GeographicID,DateID,MedicaidCharges,StandAloneCHIPCharges,TotalCosts,NetIncome,NetRevenueFromMedicaid,NetRevenueFromStandAloneCHIP,NetIncomeFromServiceToPatients, CostToChargeRatio FROM HealthCareDateInter
            WHERE RowNumber = 1
    """
    
    cursor.execute(query)
    results = cursor.fetchall()
    
    insert_query = """
        INSERT INTO HealthCareCost (HospitalID,GeographicID,DateID,MedicaidCharges,StandAloneCHIPCharges,TotalCosts,NetIncome,NetRevenueFromMedicaid,NetRevenueFromStandAloneCHIP,NetIncomeFromServiceToPatients,CostToChargeRatio)
        VALUES (?,?,?,?,?,?,?,?,?,?,?)
    """
    
    cursor.executemany(insert_query, results)
    cursor.commit()
    

## 2020 (Initial Load)

In [121]:
hospital_df_2020 = extract('./Hospital Info/CostReport_2020_Final.csv', HOSPITAL_SUBSET)
demographic_df_2020 = extract('./Demographic Info/state_M2020_dl.xlsx', DEMOGRAPHIC_SUBSET)

In [122]:
hospital_df_2020 = transform_hospital_data(hospital_df_2020)
demographic_df_2020 = transform_demographic_data(demographic_df_2020,2020)

C:\Users\Akhilesh Warty\AppData\Local\Temp\ipykernel_15704\3975306346.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
C:\Users\Akhilesh Warty\AppData\Local\Temp\ipykernel_15704\3975306346.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'#': np.nan, "*": np.nan}, inplace=True)


In [123]:
hospital_dim_2020, date_dim_2020, occupational_info_dim_2020,geographic_info_dim_2020,financial_staging_table_2020,demographic_staging_table_2020,health_care_cost_staging_table_2020 = transform_data(hospital_df_2020, demographic_df_2020,2020)

In [124]:
hospital_dim_2020.reset_index(drop=True,inplace=True)
hospital_dim_2020

,Provider_CCN,Hospital_Name
0,221300,MARTHAS VINEYARD HOSPITAL
1,111313,PUTNAM GENERAL HOSPITAL
2,341318,CHOWAN HOSPITAL INC.
3,341324,THE OUTER BANKS HOSPITAL
4,341304,BERTIE MEMORIAL HOSPITAL
...,...,...
344,110046,PIEDMONT WALTON HOSPITAL
345,60130,UCHEALTH GRANDVIEW HOSPITAL
346,260052,MERCY HOSPITAL - WASHINGTON
347,450822,MEDICAL CITY LAS COLINAS


In [125]:
geographic_info_dim_2020.reset_index(drop=True,inplace=True)
geographic_info_dim_2020.head(10)

,State_Code,County,City,RuralVsUrban
0,MA,DUKES,OAK BLUFFS,R
1,GA,PUTNAM,EATONTON,R
2,NC,CHOWAN,EDENTON,R
3,NC,DARE,NAGS HEAD,R
4,NC,BERTIE,WINDSOR,R
5,GA,CANDLER,METTER,R
6,CO,PITKIN,ASPEN,R
7,MT,CUSTER,MILES CITY,R
8,SD,BEADLE,HURON,R
9,MT,VALLEY,GLASGOW,R


In [126]:
date_dim_2020.head(10)

,Year,Quarter,Month,Day
0,2019,4,10,1
1,2020,1,1,1
2,2020,3,7,1
3,2020,3,9,1
4,2020,2,6,1
5,2020,2,4,1
6,2020,3,8,1
7,2019,4,11,1
8,2020,1,2,1
9,2020,1,2,4


In [127]:
occupational_info_dim_2020.head(10)

,NAICS,OccupationTitle,IndustryGroup,AreaName,AreaCode
0,0,All Occupations,00-0000,Alabama,AL
1,0,Management Occupations,11-0000,Alabama,AL
2,0,Chief Executives,11-1011,Alabama,AL
3,0,General and Operations Managers,11-1021,Alabama,AL
4,0,Legislators,11-1031,Alabama,AL
5,0,Advertising and Promotions Managers,11-2011,Alabama,AL
6,0,Marketing Managers,11-2021,Alabama,AL
7,0,Sales Managers,11-2022,Alabama,AL
8,0,Public Relations and Fundraising Managers,11-2030,Alabama,AL
9,0,Administrative Services and Facilities Managers,11-3010,Alabama,AL


In [128]:
verify_tables_in_database(cursor)

In [129]:
load_data_into_database(cursor, hospital_dim_2020, date_dim_2020, occupational_info_dim_2020,geographic_info_dim_2020)

Loaded all the data into the database successfully!


In [130]:
load_data_into_staging_tables(cursor, 'FinancialMeasurementStaging', financial_staging_table_2020)
load_data_into_staging_tables(cursor, 'DemographicMeasurementStaging', demographic_staging_table_2020)
load_data_into_staging_tables(cursor, 'HealthCareCostStaging', health_care_cost_staging_table_2020)

In [131]:
generate_fact_tables(cursor)

## 2021

In [132]:
hospital_df_2021 = extract('./Hospital Info/CostReport_2021_Final.csv', HOSPITAL_SUBSET)
demographic_df_2021 = extract('./Demographic Info/state_M2021_dl.xlsx', DEMOGRAPHIC_SUBSET)

In [133]:
hospital_df_2021 = transform_hospital_data(hospital_df_2021)
demographic_df_2021 = transform_demographic_data(demographic_df_2021,2021)

C:\Users\Akhilesh Warty\AppData\Local\Temp\ipykernel_15704\3975306346.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
C:\Users\Akhilesh Warty\AppData\Local\Temp\ipykernel_15704\3975306346.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'#': np.nan, "*": np.nan}, inplace=True)


In [134]:
# Take first 100 records for testing
hospital_df_2021 = hospital_df_2021.head(100)
demographic_df_2021 = demographic_df_2021.head(100)
hospital_dim_2021, date_dim_2021, occupational_info_dim_2021,geographic_info_dim_2021,financial_staging_table_2021,demographic_staging_table_2021,health_care_cost_staging_table_2021 = transform_data(hospital_df_2021, demographic_df_2021,2021)

In [135]:
verify_tables_in_database(cursor)

In [136]:
hospital_new_records,hospital_updated_records,occupational_new_records,occupational_updated_records,geographic_new_records,geographic_updated_records = load_data_into_database(cursor, hospital_dim_2021, date_dim_2021, occupational_info_dim_2021,geographic_info_dim_2021,update=True)

NEW HOSPITAL INFO RECORD : PROVIDER CCN: 51315, HOSPITAL NAME: TRINITY HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 461307, HOSPITAL NAME: HEBER VALLEY HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 450187, HOSPITAL NAME: SCOTT & WHITE HOSPITAL BRENHAM
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 271325, HOSPITAL NAME: ST LUKE COMMUNITY HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 461302, HOSPITAL NAME: MOAB REGIONAL HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 271344, HOSPITAL NAME: SIDNEY HEALTH CENTER
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 511312, HOSPITAL NAME: PRESTON MEMORIAL HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 451381, HOSPITAL NAME: CHRISTUS MFH WINNSBORO
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 451394, HOSPITAL NAME: KNOX COUNTY HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 451385, HOSPITAL NAME: GOODALL-WITCHER HOSPITAL AUTHORITY
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 111335, HOSPITAL NAME: LIBERTY REGIONAL MEDICAL CENTER
NEW HOSPITAL INFO R

In [137]:
hospital_new_records.reset_index(drop=True,inplace=True)
hospital_new_records

,Provider_CCN,Hospital_Name
0,51315,TRINITY HOSPITAL
1,461307,HEBER VALLEY HOSPITAL
2,450187,SCOTT & WHITE HOSPITAL BRENHAM
3,271325,ST LUKE COMMUNITY HOSPITAL
4,461302,MOAB REGIONAL HOSPITAL
5,271344,SIDNEY HEALTH CENTER
6,511312,PRESTON MEMORIAL HOSPITAL
7,451381,CHRISTUS MFH WINNSBORO
8,451394,KNOX COUNTY HOSPITAL
9,451385,GOODALL-WITCHER HOSPITAL AUTHORITY


In [138]:
hospital_updated_records.reset_index(drop=True,inplace=True)
hospital_updated_records

,Provider_CCN,Hospital_Name
0,60118,CENTURA ST. ANTHONY SUMMIT HOSPITAL
1,60114,CENTURA PARKER ADVENTIST HOSPITAL
2,100161,HCA FLORIDA LAKE MONROE HOSPITAL
3,60003,CENTURA LONGMONT UNITED HOSPITAL
4,60015,CENTURA ST ANTHONY HOSPITAL
5,60113,CENTURA LITTLETON ADVENTIST HOSPITAL
6,250123,SINGING RIVER GULFPORT
7,60012,CENTURA ST. MARY CORWIN HOSPITAL


In [139]:
occupational_new_records

,NAICS,OccupationTitle,IndustryGroup,AreaName,AreaCode
8,0,Public Relations Managers,11-2032,Alabama,AL
9,0,Fundraising Managers,11-2033,Alabama,AL
10,0,Administrative Services Managers,11-3012,Alabama,AL
11,0,Facilities Managers,11-3013,Alabama,AL
34,0,"Managers, All Other",11-9199,Alabama,AL
44,0,Project Management Specialists,13-1082,Alabama,AL
51,0,"Business Operations Specialists, All Other",13-1199,Alabama,AL
56,0,Financial and Investment Analysts,13-2051,Alabama,AL
59,0,Financial Risk Specialists,13-2054,Alabama,AL
64,0,"Financial Specialists, All Other",13-2099,Alabama,AL


In [140]:
occupational_updated_records

,NAICS,OccupationTitle,IndustryGroup,AreaName,AreaCode


In [141]:
geographic_new_records.reset_index(drop=True,inplace=True)
geographic_new_records

,State_Code,County,City,RuralVsUrban
0,CA,TRINITY,WEAVERVILLE,R
1,UT,WASATCH,HEBER CITY,R
2,TX,WASHINGTON,BRENHAM,R
3,MT,LAKE,RONAN,R
4,UT,GRAND,MOAB,R
5,MT,RICHLAND,SIDNEY,R
6,WV,PRESTON,KINGWOOD,U
7,TX,WOOD,WINNSBORO,R
8,TX,KNOX,KNOX CITY,R
9,TX,BOSQUE,CLIFTON,R


In [142]:
geographic_updated_records.reset_index(drop=True,inplace=True)
geographic_updated_records

,State_Code,County,City,RuralVsUrban
0,FL,SEMINOLE,SANFORD,U
1,FL,ORANGE,ORLANDO,U


In [143]:
load_data_into_staging_tables(cursor, 'FinancialMeasurementStaging', financial_staging_table_2021)
load_data_into_staging_tables(cursor, 'DemographicMeasurementStaging', demographic_staging_table_2021)
load_data_into_staging_tables(cursor, 'HealthCareCostStaging', health_care_cost_staging_table_2021)

In [144]:
generate_fact_tables(cursor)

## 2022

In [145]:
hospital_df_2022 = extract('./Hospital Info/CostReport_2022_Final.csv', HOSPITAL_SUBSET)
demographic_df_2022 = extract('./Demographic Info/state_M2022_dl.xlsx', DEMOGRAPHIC_SUBSET)

In [146]:
hospital_df_2022 = transform_hospital_data(hospital_df_2022)
demographic_df_2022 = transform_demographic_data(demographic_df_2022,2022)

C:\Users\Akhilesh Warty\AppData\Local\Temp\ipykernel_15704\3975306346.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
C:\Users\Akhilesh Warty\AppData\Local\Temp\ipykernel_15704\3975306346.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'#': np.nan, "*": np.nan}, inplace=True)


In [147]:
hospital_df_2022 = hospital_df_2022.head(100)
demographic_df_2022 = demographic_df_2022.head(100)
hospital_dim_2022, date_dim_2022, occupational_info_dim_2022,geographic_info_dim_2022,financial_staging_table_2022,demographic_staging_table_2022,health_care_cost_staging_table_2022 = transform_data(hospital_df_2022, demographic_df_2022,2022)

In [148]:
date_dim_2022

,Year,Quarter,Month,Day
0,2021,4,10,1
1,2022,1,1,1
2,2021,4,11,1
3,2021,4,12,1
4,2022,1,3,1
5,2022,1,2,1


In [149]:
verify_tables_in_database(cursor)

In [150]:
load_data_into_database(cursor, hospital_dim_2022, date_dim_2022, occupational_info_dim_2022,geographic_info_dim_2022,update=True)

NEW HOSPITAL INFO RECORD : PROVIDER CCN: 451359, HOSPITAL NAME: OCHILTREE GENERAL HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 250117, HOSPITAL NAME: HIGHLAND COMMUNITY HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 450055, HOSPITAL NAME: ROLLING PLAINS MEMORIAL HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 250019, HOSPITAL NAME: MEMORIAL HOSPITAL AT GULFPORT
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 141339, HOSPITAL NAME: TAYLORVILLE MEMORIAL HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 100156, HOSPITAL NAME: HCA FLORIDA LAKE CITY HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 451320, HOSPITAL NAME: OMNIPOINT HEALTH HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 451313, HOSPITAL NAME: FISHER COUNTY HOSPITAL DISTRICT
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 460049, HOSPITAL NAME: THE ORTHOPEDIC SPECIALTY HOSPITAL
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 670132, HOSPITAL NAME: METHODIST SOUTHLAKE MEDICAL CENTER
NEW HOSPITAL INFO RECORD : PROVIDER CCN: 451389, HOSPI

(    Provider_CCN                         Hospital_Name
 4         451359            OCHILTREE GENERAL HOSPITAL
 5         250117           HIGHLAND COMMUNITY HOSPITAL
 6         450055      ROLLING PLAINS MEMORIAL HOSPITAL
 8         250019         MEMORIAL HOSPITAL AT GULFPORT
 9         141339         TAYLORVILLE MEMORIAL HOSPITAL
 14        100156        HCA FLORIDA LAKE CITY HOSPITAL
 23        451320             OMNIPOINT HEALTH HOSPITAL
 24        451313       FISHER COUNTY HOSPITAL DISTRICT
 25        460049     THE ORTHOPEDIC SPECIALTY HOSPITAL
 31        670132    METHODIST SOUTHLAKE MEDICAL CENTER
 32        451389        PECOS COUNTY MEMORIAL HOSPITAL
 38        450039         TCHD D/B/A JPS HEALTH NETWORK
 39        330215          ROME MEMORIAL HOSPITAL  INC.
 42        160117                   THE FINLEY HOSPITAL
 45         61301                   RIO GRANDE HOSPITAL
 48        511315               POTOMAC VALLEY HOSPITAL
 53        330135        BON SECOURS COMMUNITY H

In [151]:
load_data_into_staging_tables(cursor, 'FinancialMeasurementStaging', financial_staging_table_2022)
load_data_into_staging_tables(cursor, 'DemographicMeasurementStaging', demographic_staging_table_2022)
load_data_into_staging_tables(cursor, 'HealthCareCostStaging', health_care_cost_staging_table_2022)

In [152]:
generate_fact_tables(cursor)

In [153]:
# Make a test dataframe using NAICS: 0, OCCUPATION TITLE: All Occupations, INDUSTRY GROUP: 00-0000, AREA NAME: Alabama, AREA CODE: AL
test_df = pd.DataFrame({
    'NAICS': [0],
    'OccupationTitle': ['All Occupations'],
    'IndustryGroup': ['99-9999'],
    'AreaName': ['Alabama'],
    'AreaCode': ['AL'],
})

In [154]:
test_df

,NAICS,OccupationTitle,IndustryGroup,AreaName,AreaCode
0,0,All Occupations,99-9999,Alabama,AL


In [155]:
load_data_into_table(cursor, 'OccupationalInfo', test_df, True, 2)

MATCHED OCCUPATION INFO RECORD : NAICS: 0, OCCUPATION TITLE: All Occupations, INDUSTRY GROUP: 99-9999, AREA NAME: Alabama, AREA CODE: AL
UPDATED OCCUPATION INFO RECORD : NAICS: 0, OCCUPATION TITLE: All Occupations, INDUSTRY GROUP: 99-9999, AREA NAME: Alabama, AREA CODE: AL
Records Updated in OccupationalInfo Dimension:  1


(Empty DataFrame
 Columns: [NAICS, OccupationTitle, IndustryGroup, AreaName, AreaCode]
 Index: [],
    NAICS  OccupationTitle IndustryGroup AreaName AreaCode
 0      0  All Occupations       99-9999  Alabama       AL)